In [2]:
import pandas as pd 

In [3]:
sys.path.insert(0,'../configuration/')

In [4]:
### get the config files 
import sconfig
__secretsConfig = sconfig.secretsConfig()


In [ ]:
## load the news data 
sentiNewsPath = __secretsConfig.DataLocations.sentimentscoredata
sentiNewsdf = pd.read_parquet(sentiNewsPath +'\\newsAPI' )
sentiNewsdf[(sentiNewsdf.symbol == "WMT") & (sentiNewsdf.description_compound == 0.0)].head(10)


In [76]:
(sentiNewsdf[sentiNewsdf.description_compound > 0.0].groupby(['symbol','NewsDate']).size()/sentiNewsdf.groupby(['symbol','NewsDate']).size()).reset_index()

,symbol,NewsDate,0
0,AAPL,2022-09-06,NaN
1,AAPL,2022-09-07,0.666667
2,AAPL,2022-09-16,0.333333
3,AAPL,2022-09-20,1.000000
4,AAPL,2022-09-26,NaN
...,...,...,...
527,WMT,2022-11-14,1.000000
528,WMT,2022-11-15,0.500000
529,WMT,2022-11-16,NaN
530,WMT,2022-11-18,NaN


In [7]:
sentiNewsdf["NewsDate"] = pd.to_datetime(sentiNewsdf["publishedAt"]).dt.date
sentiNewsdf["NewsDate"] = pd.to_datetime(sentiNewsdf["NewsDate"])
sentiGrouped = sentiNewsdf.groupby(['symbol','NewsDate']).mean().reset_index()
#sentiGrouped= sentiGrouped.drop(['id','article_id'],axis=1)
##sentiGrouped
sentiGrouped.tail(2)

,symbol,NewsDate,title_neg,title_neu,title_pos,title_compound,description_neg,description_neu,description_pos,description_compound
530,WMT,2022-11-18,0.000,0.588,0.412,0.5994,0.00,0.00,0.0,0.0000
531,WMT,2022-11-19,0.427,0.573,0.000,-0.8807,0.06,0.94,0.0,-0.2263


In [78]:
sentiGrouped['sentiPositiveNews'] = (sentiNewsdf[sentiNewsdf.description_compound > 0.0].groupby(['symbol','NewsDate']).size()/sentiNewsdf.groupby(['symbol','NewsDate']).size()).reset_index()[0]
sentiGrouped['sentiNeutralNews'] = (sentiNewsdf[sentiNewsdf.description_compound == 0.0].groupby(['symbol','NewsDate']).size()/sentiNewsdf.groupby(['symbol','NewsDate']).size()).reset_index()[0]
sentiGrouped['sentiNegativeNews'] = (sentiNewsdf[sentiNewsdf.description_compound < 0.0].groupby(['symbol','NewsDate']).size()/sentiNewsdf.groupby(['symbol','NewsDate']).size()).reset_index()[0]

sentiGrouped

,symbol,NewsDate,title_neg,title_neu,title_pos,title_compound,description_neg,description_neu,description_pos,description_compound,sentiPositiveNews,sentiNeutralNews,sentiNegativeNews
0,AAPL,2022-09-06,0.375000,0.625000,0.000000,-0.585900,0.2540,0.746000,0.000000,-0.585900,NaN,NaN,1.000000
1,AAPL,2022-09-07,0.000000,0.733667,0.266333,0.598400,0.0000,0.852333,0.147667,0.435800,0.666667,0.333333,NaN
2,AAPL,2022-09-16,0.082333,0.808333,0.109333,0.003633,0.0920,0.856333,0.051667,-0.172633,0.333333,NaN,0.666667
3,AAPL,2022-09-20,0.166000,0.662000,0.172000,0.025800,0.0000,0.880000,0.120000,0.381800,1.000000,NaN,NaN
4,AAPL,2022-09-26,0.000000,1.000000,0.000000,0.000000,0.1620,0.838000,0.000000,-0.401900,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,WMT,2022-11-14,0.000000,1.000000,0.000000,0.000000,0.0000,0.923000,0.077000,0.440400,1.000000,NaN,NaN
528,WMT,2022-11-15,0.100000,0.732500,0.167500,-0.043150,0.0690,0.840000,0.091000,0.029300,0.500000,NaN,0.500000
529,WMT,2022-11-16,0.000000,0.770000,0.230000,0.429350,0.0265,0.938000,0.035500,-0.041400,NaN,0.500000,0.500000
530,WMT,2022-11-18,0.000000,0.588000,0.412000,0.599400,0.0000,0.000000,0.000000,0.000000,NaN,1.000000,NaN


In [79]:
## load the stock news
stockNewsPath = __secretsConfig.DataLocations.djiatickerdata
stockNewsDF = pd.read_parquet(stockNewsPath)

## add the returns columns
stockNewsDF['return'] = stockNewsDF.groupby('symbol')['Close'].pct_change().mul(100)
stockNewsDF = stockNewsDF.reset_index()
stockNewsDF.head(2)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,companyname,symbol,return
0,1980-12-12,0.099874,0.100308,0.099874,0.099874,469033600,0.0,0.0,Apple Inc,AAPL,NaN
1,1980-12-15,0.095098,0.095098,0.094663,0.094663,175884800,0.0,0.0,Apple Inc,AAPL,-5.217057


In [ ]:
## merge the news and stock data 
mergedDF = pd.merge(stockNewsDF,sentiGrouped,how="inner", left_on=["Date","symbol"], right_on=["NewsDate","symbol"])
mergedDF['month_year'] = pd.to_datetime(mergedDF.Date).dt.to_period('M')
mergedDF = mergedDF[["symbol","Date","month_year","Close","return","description_compound","sentiPositiveNews","sentiNeutralNews","sentiNegativeNews"]]

In [110]:
mergedDF[(mergedDF.month_year == '2022-09') & (mergedDF.symbol == 'AAPL')]

,symbol,Date,month_year,Close,return,description_compound,sentiPositiveNews,sentiNeutralNews,sentiNegativeNews
0,AAPL,2022-09-06,2022-09,154.274078,-0.821512,-0.585900,NaN,NaN,1.000000
1,AAPL,2022-09-07,2022-09,155.701721,0.925394,0.435800,0.666667,0.333333,NaN
2,AAPL,2022-09-16,2022-09,150.450424,-1.096008,-0.172633,0.333333,NaN,0.666667
3,AAPL,2022-09-20,2022-09,156.640152,1.566551,0.381800,1.000000,NaN,NaN
4,AAPL,2022-09-26,2022-09,150.520309,0.226025,-0.401900,NaN,NaN,1.000000
5,AAPL,2022-09-29,2022-09,142.244034,-4.911905,-0.226300,NaN,NaN,1.000000
6,AAPL,2022-09-30,2022-09,137.971115,-3.003935,0.493900,1.000000,NaN,NaN


In [136]:
## Monthly Merging Data 
monthlyMergedDF = mergedDF.groupby(["month_year","symbol"]).agg(
    {
        'return':['mean'],
        'description_compound' :['mean'],
        'sentiPositiveNews' : ['sum'],
        'sentiNeutralNews' : ['sum'],
        'sentiNegativeNews' : ['sum'],
        'month_year' : ['count']
    }
)
monthlyMergedDF = monthlyMergedDF.reset_index()
## Easiest way to convert the multiindex to single single index 
monthlyMergedDF.columns = ['month_year','symbol','return_monthly_average','sentiment_monthly_average','monthly_positive_news','monthly_neutral_news','monthly_negative_news','record_count']
monthlyMergedDF["monthly_positive_news"] = monthlyMergedDF.monthly_positive_news / monthlyMergedDF.record_count
monthlyMergedDF["monthly_neutral_news"] = monthlyMergedDF.monthly_neutral_news / monthlyMergedDF.record_count
monthlyMergedDF["monthly_negative_news"] = monthlyMergedDF.monthly_negative_news / monthlyMergedDF.record_count
monthlyMergedDF['month_year'] = monthlyMergedDF.month_year.astype(str)
monthlyMergedDF.to_parquet("D:\\data\\development\\FinAnalytics\\FinancialAnalytics_Bits\\stocks_data\\analytics\\stock_senti_corr\\data.parquet",engine='fastparquet')


#monthlyMergedDF.head(10)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\data\\development\\FinAnalytics\\FinancialAnalytics_Bits\\stocks_data\\stock_senti_corr\\data.parquet'

In [ ]:
mergedDF.where(mergedDF.symbol == "AAPL").dropna()

In [23]:
## getting the correlation between returns and news 
corr_returns_title_apple = mergedDF[mergedDF.symbol.eq('NKE')]["return"].corr(mergedDF["title_compound"])
corr_returns_description_apple = mergedDF[mergedDF.symbol.eq('NKE')]["return"].corr(mergedDF["description_compound"])
print(corr_returns_title_apple)
print(corr_returns_description_apple)

0.20861506790019746
-0.09276555610234087
